In [6]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import sys
import os

# ==============================
# PATH & DB CONFIG
# ==============================

sys.path.append(os.path.abspath("src"))
from db_config import DB_URI

# ==============================
# CONFIG
# ==============================

OUTPUT_DIR = "eda/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

engine = create_engine(DB_URI)

# ==============================
# LOAD DATA (GOLD)
# ==============================

df = pd.read_sql("""
    SELECT *
    FROM gold.dashboard_rides
""", engine)

# Ensure date format
df["ride_date"] = pd.to_datetime(df["ride_date"])

# ==============================
# 1️⃣ DAILY RIDE VOLUME
# ==============================

rides_per_day = (
    df.groupby("ride_date")
      .size()
      .reset_index(name="total_rides")
)

plt.figure()
plt.plot(rides_per_day["ride_date"], rides_per_day["total_rides"])
plt.xlabel("Ride Date")
plt.ylabel("Total Rides")
plt.title("Daily Ride Volume Trend")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/daily_ride_volume.png")
plt.close()

# ==============================
# 2️⃣ RIDE STATUS DISTRIBUTION
# ==============================

ride_status_counts = df["ride_status"].value_counts()

plt.figure()
plt.bar(ride_status_counts.index, ride_status_counts.values)
plt.xlabel("Ride Status")
plt.ylabel("Number of Rides")
plt.title("Ride Status Distribution")
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/ride_status_distribution.png")
plt.close()

# ==============================
# 3️⃣ AVG WAITING TIME BY CITY
# ==============================

avg_wait_city = (
    df.groupby("ride_city")["waiting_time_min"]
      .mean()
      .sort_values(ascending=False)
)

plt.figure()
plt.bar(avg_wait_city.index, avg_wait_city.values)
plt.xlabel("City")
plt.ylabel("Avg Waiting Time (min)")
plt.title("Average Waiting Time by City")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/avg_waiting_time_by_city.png")
plt.close()

# ==============================
# 4️⃣ FARE VS DISTANCE
# ==============================

plt.figure()
plt.scatter(df["distance_km"], df["fare_amount"])
plt.xlabel("Distance (km)")
plt.ylabel("Fare Amount")
plt.title("Fare vs Distance")
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/fare_vs_distance.png")
plt.close()

# ==============================
# 5️⃣ PAYMENT METHOD DISTRIBUTION
# ==============================

payment_counts = df["payment_method"].value_counts()

plt.figure()
plt.pie(
    payment_counts.values,
    labels=payment_counts.index,
    autopct="%1.1f%%",
    startangle=90
)
plt.title("Payment Method Distribution")
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/payment_method_distribution.png")
plt.close()

print("✅ EDA completed. Charts saved in eda/outputs/")


ValueError: invalid literal for int() with base 10: 'None'